# Thought Process
-> 5 genres are there with 1000 articles in total (taken for sample) </br>
-> bandit chooses 1 article </br>
-> reward is based on the genre of article chosen, whether it matches with the users </br>
-> if match, give points, if not match, can put 0 or negative </br>

In [8]:
import numpy as np
import pandas as pd

In [9]:
GENRES = [i for i in range(5)]

In [10]:
NUM_ARTICLES = 1000
article = [np.random.choice(GENRES, p = [0.5, 0.2, 0.1, 0.15, 0.05]) for _ in range(NUM_ARTICLES)]
article = pd.DataFrame(article, columns=["genre"])
article.to_csv("article.csv", index=False)

In [11]:
article.value_counts()

genre
0        459
1        222
3        167
2        105
4         47
Name: count, dtype: int64

In [12]:
user_choices = article.genre
reward_correct = 2
reward_incorrect = -1

In [13]:
class MaB():
    def __init__(self, n_arms, eps = 0.9, decay = 0.95):
        self.n_arms = n_arms
        self.rewards = np.zeros(n_arms)
        self.counts = np.zeros(n_arms)
        self.eps = eps
        self.decay = decay

    def select(self):
        if np.random.rand() < self.eps:
            return np.random.randint(self.n_arms)
        else:
            return np.argmax(self.rewards / (self.counts + 1))
        
    def update(self, arm, reward):
        self.counts[arm] += 1
        self.rewards[arm] = self.rewards[arm] * self.decay + reward
        self.eps *= 0.99

    def output(self):
        arm = self.select()
        return arm
    
    def print_counts(self):
        return self.counts

In [14]:
n_arms = 5
mab = MaB(n_arms)

for i in range(100):
    out = mab.output()
    print(mab.rewards)
    print(mab.counts)
    if (out == user_choices[i]):
        reward = reward_correct
    else:
        reward = reward_incorrect
    mab.update(out, reward)
    print(out)

[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
1
[ 0. -1.  0.  0.  0.]
[0. 1. 0. 0. 0.]
3
[ 0. -1.  0. -1.  0.]
[0. 1. 0. 1. 0.]
0
[ 2. -1.  0. -1.  0.]
[1. 1. 0. 1. 0.]
2
[ 2. -1. -1. -1.  0.]
[1. 1. 1. 1. 0.]
3
[ 2.   -1.   -1.   -1.95  0.  ]
[1. 1. 1. 2. 0.]
4
[ 2.   -1.   -1.   -1.95 -1.  ]
[1. 1. 1. 2. 1.]
2
[ 2.   -1.   -1.95 -1.95 -1.  ]
[1. 1. 2. 2. 1.]
1
[ 2.   -1.95 -1.95 -1.95 -1.  ]
[1. 2. 2. 2. 1.]
0
[ 3.9  -1.95 -1.95 -1.95 -1.  ]
[2. 2. 2. 2. 1.]
4
[ 3.9  -1.95 -1.95 -1.95 -1.95]
[2. 2. 2. 2. 2.]
4
[ 3.9    -1.95   -1.95   -1.95   -2.8525]
[2. 2. 2. 2. 3.]
0
[ 2.705  -1.95   -1.95   -1.95   -2.8525]
[3. 2. 2. 2. 3.]
3
[ 2.705  -1.95   -1.95   -2.8525 -2.8525]
[3. 2. 2. 3. 3.]
0
[ 4.56975 -1.95    -1.95    -2.8525  -2.8525 ]
[4. 2. 2. 3. 3.]
2
[ 4.56975 -1.95    -2.8525  -2.8525  -2.8525 ]
[4. 2. 3. 3. 3.]
2
[ 4.56975  -1.95     -3.709875 -2.8525   -2.8525  ]
[4. 2. 4. 3. 3.]
4
[ 4.56975  -1.95     -3.709875 -2.8525   -3.709875]
[4. 2. 4. 3. 4.]
0
[ 6.3412625 -1.95      -3.709875  -2.